In [ ]:
from google.colab import files
files.upload()

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c dogs-vs-cats

 99% 801M/812M [00:16<00:00, 52.7MB/s]
100% 812M/812M [00:16<00:00, 51.0MB/s]


In [4]:
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

In [ ]:
import os, pathlib, shutil

n_path = pathlib.Path("dogs-vs-cats-small")
o_path = pathlib.Path("train")

def make_subsets(name, start, end):
    for category in ("dog", "cat"):
        dir = n_path / name / category
        os.makedirs(dir)

        fnames = [f"{category}.{i}.jpg" for i in range(start, end)]

        for fname in fnames:
            shutil.copyfile(src=o_path / fname, dst=dir / fname)

make_subsets("train", 0, 1_000)
make_subsets("val", 1_000, 1_500)
make_subsets("test", 1_500, 2_500)

In [7]:
from tensorflow.keras.utils import image_dataset_from_directory

train_data = image_dataset_from_directory(
    n_path / "train",
    image_size=(180, 180),
    batch_size=32
)

val_data = image_dataset_from_directory(
    n_path / "val",
    image_size=(180, 180),
    batch_size=32
)

test_data = image_dataset_from_directory(
    n_path / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [14]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, RandomFlip, RandomRotation, RandomZoom, Dense, Flatten, Dropout
from tensorflow.keras.applications import efficientnet_v2

data_aug = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.2)
])

conv_base = efficientnet_v2.EfficientNetV2B2(
    include_top=False,
    weights="imagenet"
)

conv_base.trainable = False

inputs = Input(shape=(180, 180, 3))
x = data_aug(inputs)
x = conv_base(x)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint

callbacks=[
    ModelCheckpoint(
        filepath="effnet.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

In [15]:
res = model.fit(train_data,
                epochs=20,
                validation_data=val_data,
                callbacks=callbacks
)

Epoch 1/20
63/63 [==============================] - 17s 142ms/step - loss: 0.4151 - accuracy: 0.9595 - val_loss: 0.2666 - val_accuracy: 0.9790
Epoch 2/20
63/63 [==============================] - 7s 107ms/step - loss: 0.3219 - accuracy: 0.9695 - val_loss: 0.2795 - val_accuracy: 0.9830
Epoch 3/20
63/63 [==============================] - 7s 106ms/step - loss: 0.2549 - accuracy: 0.9830 - val_loss: 0.2674 - val_accuracy: 0.9860
Epoch 4/20
63/63 [==============================] - 7s 105ms/step - loss: 0.2317 - accuracy: 0.9840 - val_loss: 0.2657 - val_accuracy: 0.9910
Epoch 5/20
63/63 [==============================] - 7s 105ms/step - loss: 0.3010 - accuracy: 0.9850 - val_loss: 0.3340 - val_accuracy: 0.9840
Epoch 6/20
63/63 [==============================] - 7s 106ms/step - loss: 0.1119 - accuracy: 0.9900 - val_loss: 0.3620 - val_accuracy: 0.9850
Epoch 7/20
63/63 [==============================] - 7s 104ms/step - loss: 0.4033 - accuracy: 0.9840 - val_loss: 0.5852 - val_accuracy: 0.9850
Epoch

In [16]:
from tensorflow.keras.models import load_model

test_model = load_model("effnet.keras")

loss, acc = test_model.evaluate(test_data)
print(f"Accuracy: {acc}")

63/63 [==============================] - 6s 61ms/step - loss: 0.0880 - accuracy: 0.9865
Accuracy: 0.9865000247955322
